In [ ]:
import numpy as np
import random
import gensim
from nltk.corpus import words
from functools import reduce
import heapq
import os
import collections
from sklearn.cluster import KMeans
from sklearn import metrics
import copy

In [ ]:
# loads glove into gensim
model = gensim.models.KeyedVectors.load_word2vec_format('glove.6B.300d.w2vformat.txt', binary=False)

In [ ]:
# cleans codename word list
words_upper = ["Hollywood", "Well", "Foot", "NewYork", "Spring", "Court", "Tube", "Point", "Tablet", "Slip", "Date", "Drill", "Lemon", "Bell", "Screen", "Fair", "Torch", "State", "Match", "Iron", "Block", "France", "Australia", "Limousine", "Stream", "Glove", "Nurse", "Leprechaun", "Play", "Tooth", "Arm", "Bermuda", "Diamond", "Whale", "Comic", "Mammoth", "Green", "Pass", "Missile", "Paste", "Drop", "Pheonix", "Marble", "Staff", "Figure", "Park", "Centaur", "Shadow", "Fish", "Cotton", "Egypt", "Theater", "Scale", "Fall", "Track", "Force", "Dinosaur", "Bill", "Mine", "Turkey", "March", "Contract", "Bridge", "Robin", "Line", "Plate", "Band", "Fire", "Bank", "Boom", "Cat", "Shot", "Suit", "Chocolate", "Roulette", "Mercury", "Moon", "Net", "Lawyer", "Satellite", "Angel", "Spider", "Germany", "Fork", "Pitch", "King", "Crane", "Trip", "Dog", "Conductor", "Part", "Bugle", "Witch", "Ketchup", "Press", "Spine", "Worm", "Alps", "Bond", "Pan", "Beijing", "Racket", "Cross", "Seal", "Aztec", "Maple", "Parachute", "Hotel", "Berry", "Soldier", "Ray", "Post", "Greece", "Square", "Mass", "Bat", "Wave", "Car", "Smuggler", "England", "Crash", "Tail", "Card", "Horn", "Capital", "Fence", "Deck", "Buffalo", "Microscope", "Jet", "Duck", "Ring", "Train", "Field", "Gold", "Tick", "Check", "Queen", "Strike", "Kangaroo", "Spike", "Scientist", "Engine", "Shakespeare", "Wind", "Kid", "Embassy", "Robot", "Note", "Ground", "Draft", "Ham", "War", "Mouse", "Center", "China", "Bolt", "Spot", "Piano", "Pupil", "Plot", "Lion", "Police", "Head", "Litter", "Concert", "Mug", "Vacuum", "Atlantis", "Straw", "Switch", "Skyscraper", "Laser", "Scuba Diver", "Africa", "Plastic", "Dwarf", "Lap", "Life", "Honey", "Horseshoe", "Unicorn", "Spy", "Pants", "Wall", "Paper", "Sound", "Ice", "Tag", "Web", "Fan", "Orange", "Temple", "Canada", "Scorpion", "Undertaker", "Mail", "Europe", "Soul", "Apple", "Pole", "Tap", "Mouth", "Ambulance", "Dress", "IceCream", "Rabbit", "Buck", "Agent", "Sock", "Nut", "Boot", "Ghost", "Oil", "Superhero", "Code", "Kiwi", "Hospital", "Saturn", "Film", "Button", "Snowman", "Helicopter", "Log", "Princess", "Time", "Cook", "Revolution", "Shoe", "Mole", "Spell", "Grass", "Washer", "Game", "Beat", "Hole", "Horse", "Pirate", "Link", "Dance", "Fly", "Pit", "Server", "School", "Lock", "Brush", "Pool", "Star", "Jam", "Organ", "Berlin", "Face", "Luck", "Amazon", "Cast", "Gas", "Club", "Sink", "Water", "Chair", "Shark", "Jupiter", "Copper", "Jack", "Platypus", "Stick", "Olive", "Grace", "Bear", "Glass", "Row", "Pistol", "London", "Rock", "Van", "Vet", "Beach", "Charge", "Port", "Disease", "Palm", "Moscow", "Pin", "Washington", "Pyramid", "Opera", "Casino", "Pilot", "String", "Night", "Chest", "Yard", "Teacher", "Pumpkin", "Thief", "Bark", "Bug", "Mint", "Cycle", "Telescope", "Calf", "Air", "Box", "Mount", "Thumb", "Antarctica", "Trunk", "Snow", "Penguin", "Root", "Bar", "File", "Hawk", "Battery", "Compound", "Slug", "Octopus", "Whip", "America", "Ivory", "Pound", "Sub", "Cliff", "Lab", "Eagle", "Genius", "Ship", "Dice", "Hood", "Heart", "Novel", "Pipe", "Himalayas", "Crown", "Round", "India", "Needle", "Shop", "Watch", "Lead", "Tie", "Table", "Cell", "Cover", "Czech", "Back", "Bomb", "Ruler", "Forest", "Bottle", "Space", "Hook", "Doctor", "Ball", "Bow", "Degree", "Rome", "Plane", "Giant", "Nail", "Dragon", "Stadium", "Flute", "Carrot", "Wake", "Fighter", "Model", "Tokyo", "Eye", "Mexico", "Hand", "Swing", "Key", "Alien", "Tower", "Poison", "Cricket", "Cold", "Knife", "Church", "Board", "Cloak", "Ninja", "Olympus", "Belt", "Light", "Death", "Stock", "Millionaire", "Day", "Knight", "Pie", "Bed", "Circle", "Rose", "Change", "Cap", "Triangle", "Chick"]
words = [x.lower() for x in words_upper]
for word in words :
    if ' ' in word :
        words.remove(word)


# generate new board
def new_game(words) :
    board = random.sample(words, 25)

    p1 = board[:9]
    p2 = board[9:17]
    neu = board[17:24]
    assassin = [board[24]]
    
    # get vectors for each word from the model
    p1_vecs = model[p1]
    p2_vecs = model[p2]
    assassin_vec = model[assassin]
    
    return board, p1, p2, neu, assassin, p1_vecs, p2_vecs, assassin_vec

# k means clustering
def clustering(vecs, n=5) :
    initial = KMeans(n_clusters=n)
    clusters = initial.fit_predict(vecs)
    centroids = initial.cluster_centers_
    
    # finding cluster size and tightness (mean distance from centroid)
    # https://stackoverflow.com/questions/40828929/sklearn-mean-distance-from-centroid-of-each-cluster
    mean_dists = {}
    mean_count = {}
    for i in range(n):
        mean_dists[i] = 0
        mean_count[i] = 0
    
    for i in range(len(vecs)) :
        cluster = clusters[i]
        centroid = centroids[cluster]
        dist = np.linalg.norm(centroid-p1_vecs[i])
        mean_dists[cluster] += dist
        mean_count[cluster] += 1
    
    mean_of_cluster = []
    for i in range(n):
        mean_of_cluster.append(mean_dists[i]/mean_count[i])
    
    return clusters, mean_count, mean_of_cluster

# find the largest cluster from list of clusters
# within a tightness level set by hyperparameters 
def tightest_cluster(clusters_list, mean_count_list, tightness_list, player) :
    
    # hyperparams
    cluster_six_cutoff = 6.3
    cluster_five_cutoff = 6
    cluster_four_cutoff = 5.7
    cluster_rest_cutoff = 5.5
    
    current_max = None
    current_count = 0
    turn_max = 0
    largest_clusters = []
    
    # finds largest/tightest of all three times we ran kmeans
    for turn in range(3):
        clusters = clusters_list[turn]
        mean_count = mean_count_list[turn]
        tightness = tightness_list[turn]
        
        clusters_by_size = []
        max_i = 0
        current_size = 0

        for i in range(len(mean_count)):
            clusters_by_size.append((tightness[i], mean_count[i], i))

        # sort by largest size
        sorted_clusters_by_size = sorted(clusters_by_size, key=lambda tup: tup[1], reverse=True)
        
        for i, cluster in enumerate(sorted_clusters_by_size):
            
            # keep a list of largest sized clusters in case no cluster is within hyperparams
            if i == 0:
                largest_clusters.append((cluster, turn))
                
            # if larger than current largest cluster, make it our target cluster if it fits within hyperparams
            if cluster[1] > current_count:
                if (cluster[1] >= 6 and cluster[0] < cluster_six_cutoff) or (cluster[1] >= 5 and cluster[0] < cluster_five_cutoff) or (cluster[1] >= 4 and cluster[0] < cluster_four_cutoff) or cluster[0] < cluster_rest_cutoff:
                    current_max = cluster
                    current_count = cluster[1]
                    turn_max = turn
                    break
                    
            # if same size as current largest cluster, check if tighter
            elif cluster[1] == current_count:
                if cluster[0] < current_max[0]:
                    current_max = cluster
                    current_count = cluster[1]
                    turn_max = turn
                    break
    
    # if no cluster is within hyperparams, choose tightest of the largest cluster from the three iterations
    if current_max == None:
        largest_clusters_sorted = sorted(largest_clusters, key=lambda tup: tup[0][0])
        current_max = largest_clusters_sorted[0][0]
        current_count = largest_clusters_sorted[0][0][1]
        turn_max = largest_clusters_sorted[0][1]
        
    res = []
    
    # change clusters into words
    for i in range(len(clusters_list[turn_max])) :
        if clusters_list[turn_max][i] == current_max[2] :
            res.append(player[i])
    
    # returns number of words and response
    return len(res), res

# spymaster gives hint based on word cluster, opponent's words, and assassin
def give_hint(pos, neg, restriction=50000):
    full_hint = model.most_similar(positive=pos, negative=neg, restrict_vocab=restriction)
    index = 0
    i = 0
    
    # make sure that the hint is allowed (does not include substrings of words on the board)
    while i < len(board):
        if full_hint[index][0] in board[i] or board[i] in full_hint[index][0] or full_hint[index][0][:-1] in board[i] or board[i][:-1] in full_hint[index][0]:
            index += 1
            i = -1
        i += 1
        
    return full_hint[index][0]

# guesser agent compares remaining words on board with hint, and returns number of words with highest similarities

def new_guesser(player_q, turn, hint, num_words) :
    new_q = []
    similarities = []
    unweighted = []
    guesser = []
    
    # get top guesses of words (up to the number that the spymaster gave)
    for item in player_q :
        similarity = model.similarity(hint, item[0])
        similarities.append((item[0], similarity))
        
    sort_by_similarity = sorted(similarities, key=lambda tup: tup[1], reverse=True)
    top_guesses = sort_by_similarity[:num_words]
    
    # if it's the team's turn, add the top guesses' similarity levels to the current q values
    if turn :
        for item in player_q :
            added = False
            for guess in top_guesses:
                
                # if a top guess, add similarity value to current value
                if item[0] == guess[0]:
                    guesser.append((item[0], item[1] + 1.75 * guess[1]))
                    new_prob = item[1] + guess[1]
                    new_q.append((item[0], new_prob))
                    added = True
                    
            # if not a top guess, keep previous values
            if added == False:
                new_q.append((item[0], item[1]))
                guesser.append((item[0], item[1]))
    
    # if it's the opposing team's turn, subtract these top guess similarity values from your own q values
    else :        
        for item in player_q :
            added = False
            for guess in top_guesses:
                
                # if a top guess, subtrack similarity value to current value
                if item[0] == guess[0]:
                    new_prob = item[1] - guess[1]
                    new_q.append((item[0], new_prob))
                    added = True
                    
            # if not a top guess, keep previous values
            if added == False:
                new_q.append((item[0], item[1]))
        
    new_q = sorted(new_q, key=lambda tup: tup[1], reverse=True)
    guesser_sorted = sorted(guesser, key=lambda tup: tup[1], reverse=True)
    
    # return the edited q, as well as the list of words to guess sorted by highest cumulative similarity values
    return new_q, guesser_sorted

# Taking a turn
def take_turn(board, p1, p2, p1_q, p2_q, assassin, turn) :
    
    if turn:
        print " "
        print "Team 1's turn. Words left for this team are:", p1
    else:
        print " "
        print "Team 2's turn. Words left for this team are:", p2
    
    # alternate turns between teams
    p2_turn = not turn
    
    # remove from q words that are no longer on the board
    new_q1 = []
    new_q2 = []
    for item in p1_q :
        if item[0] in board :
            new_q1.append(item)
    
    for item2 in p2_q :
        if item2[0] in board :
            new_q2.append(item2)
    
    p1_q = new_q1
    p2_q = new_q2
    
    if turn :
        player = p1
        player_vecs = model[p1]
        n = max(len(p1)/2, 2)
    else :
        player = p2
        player_vecs = model[p2]
        n = max(len(p2)/2, 2)
        
    # spymaster clustering based on remaining words
    clusters_list = []
    mean_count_list = []
    mean_of_cluster_list = []
    
    # we run kmeans three times, one with n - 1 clusters (or the number of words left on the team, whichever is smaller),
    # one with n clusters, and one with n + 1 clusters
    for i in range(3):
        clusters_temp, mean_count_temp, mean_of_cluster_temp = clustering(player_vecs, min(n - 1 + i, len(player)))
        clusters_list.append(clusters_temp)
        mean_count_list.append(mean_count_temp)
        mean_of_cluster_list.append(mean_of_cluster_temp)
    
    # get the largest/tightest cluster and number of words in the cluster
    num_words, tightest = tightest_cluster(clusters_list, mean_count_list, mean_of_cluster_list, player)
    
    hint = give_hint(tightest, assassin, restriction=50000)
        
    print "Hint:", hint
    print "Number:", num_words
    print "Words the spymaster is going for", tightest
    
    # update similarity values in the q for each team
    p1_q, guesses1 = new_guesser(p1_q, turn, hint, num_words)
    p2_q, guesses2 = new_guesser(p2_q, p2_turn, hint, num_words)
    
    # guess words up to a cutoff value for similarity
    cutoff = 0.1
    
    if turn :
        guesses = []
        number = min(len(guesses1), range(num_words))
        for num in range(number):
            if guesses1[num][1] > cutoff:
                guesses.append(guesses1[num])

    else :
        guesses = []
        number = min(len(guesses2), range(num_words))
        for num in range(number):
            if guesses2[num][1] > cutoff:
                guesses.append(guesses2[num])
                
    actually_guessed_words = []
    incorrect_guesses = 0
    turn_end = False
    
    for i in range(len(guesses)) :
        if assassin[0] == guesses[i][0] :
            incorrect_guesses += 1
            if turn :
                string = "Team 1 guessed '"+ str(guesses[i][0])+ "', which was the assassin word :( Team 2 wins."
                return string, board, p1_q, p2_q, incorrect_guesses
            else : 
                string = "Team 2 guessed '"+ str(guesses[i][0])+ "', which was the assassin word :( Team 1 wins."
                return string, board, p1_q, p2_q, incorrect_guesses
        
        board.remove(guesses[i][0])
        actually_guessed_words.append(guesses[i][0])
        
        if guesses[i][0] in p2 :
            p2.remove(guesses[i][0])
            if turn :
                incorrect_guesses += 1
                print "Correctly guessed words:", actually_guessed_words[:-1]
                print "Oops! Incorrectly guessed a word from team 1:", guesses[i][0]
                turn_end = True
                break
        elif guesses[i][0] in p1 :
            p1.remove(guesses[i][0])
            if not turn :
                incorrect_guesses += 1
                print "Correctly guessed words:", actually_guessed_words[:-1]
                print "Oops! Incorrectly guessed a word from team 2:", guesses[i][0]
                turn_end = True
                break
        else:
            incorrect_guesses += 1
            print "Correctly guessed words:", actually_guessed_words[:-1]
            print "Oops! Incorrectly guessed a neutral word:", guesses[i][0]
            turn_end = True
            break
    
    if turn_end == False:
        print "Correctly guessed words:", actually_guessed_words

    if not p1 :
        return "Team 1 wins!", board, p1_q, p2_q, incorrect_guesses
    elif not p2 :
        return "Team 2 wins!", board, p1_q, p2_q, incorrect_guesses
    else :
        return "continue", board, p1_q, p2_q, incorrect_guesses


In [ ]:
# play a game!
success = []
num_turns_to_win = []
avg_wrong = []

# create board
board, p1, p2, neu, assassin, p1_vecs, p2_vecs, assassin_vec = new_game(words)
print "This game's board is:", board
print "Team 1's words are:", p1
print "Team 2's words are:", p2
print "The assasin word is:", assassin
p1_q = []
p2_q = []

# initialize priority queues
for i in range(25) :
    p1_q.append((board[i], 0.0))
    p2_q.append((board[i], 0.0))

# kepping track of # of incorrect guesses
incorrect_guesses = 0

game_end = "continue"
turn_number = 1
incorrect_guesses = 0

while game_end == "continue" :
    game_end, board, p1_q, p2_q, wrong = take_turn(board, p1, p2, p1_q, p2_q, assassin, turn_number%2)
    incorrect_guesses += wrong
    turn_number +=1 

num_turns_to_win.append(turn_number)
avg_wrong.append(incorrect_guesses)

print " "
print game_end
print "Total turns taken by both teams:", np.mean(num_turns_to_win)
print "Total wrong guesses:", np.mean(avg_wrong)